In [1]:
from pennylane import numpy as np
import matplotlib as mpl

import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from pennylane import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import pennylane as qml
import seaborn as sns
from sklearn.svm import SVC
import matplotlib as mpl
import warnings

np.random.seed(1359)

In [13]:
iris_data = load_iris()
X = iris_data.data
Y = iris_data.target
classes = [0,1,2]
n_classes = len(classes)

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.7, random_state=42, stratify=Y)
X_train.shape, X_test.shape

((45, 4), (105, 4))

In [15]:
import pennylane as qml

def layer(x, params, wires, i0=0, inc=1):
    """Building block of the embedding ansatz"""
    z1 = x*params[0] + params[1]
    qml.RY(z1[0], 0)
    qml.RZ(z1[1], 0)
    qml.RY(z1[0], 0)
    qml.RZ(z1[1], 0)
    
    z2 = x*params[2] + params[3]
    qml.RY(z1[0], 1)
    qml.RZ(z1[1], 1)
    qml.RY(z2[0], 1)
    qml.RZ(z2[1], 1)
    
    qml.CNOT((0,1))

In [16]:
def ansatz(x, params, wires):
    """The embedding ansatz"""
    layer(x, params, wires)

adjoint_ansatz = qml.adjoint(ansatz)

def random_params():
    """Generate random variational parameters in the shape for the ansatz."""
    return np.random.uniform(0, 2 * np.pi, (4, 4), requires_grad=True)

In [17]:
dev = qml.device("default.qubit", wires=2, shots=None)
wires = dev.wires.tolist()

In [18]:
@qml.qnode(dev)
def kernel_circuit(x1, x2, params):
    ansatz(x1, params, wires=wires)
    adjoint_ansatz(x2, params, wires=wires)
    return qml.probs(wires=wires)

def kernel(x1, x2, params):
    return kernel_circuit(x1, x2, params)[0]

In [19]:
params = [random_params() for _ in classes]

In [20]:
print(qml.draw(kernel_circuit)(X_train[0], X_train[1], params[0]))

0: ──RY(3.66)──RZ(0.18)──RY(3.66)──RZ(0.18)─╭●─╭●───RZ(0.20)†──RY(6.37)†──RZ(0.20)†──RY(6.37)†─┤
1: ──RY(3.66)──RZ(0.18)──RY(6.17)──RZ(5.69)─╰X─╰X†──RZ(5.83)†──RY(7.82)†──RZ(0.20)†──RY(6.37)†─┤

  ╭Probs
  ╰Probs


In [21]:
kernel_value = kernel(X_train[0], X_train[1], params[0])
print(f"The kernel value between the first and second datapoint is {kernel_value:.3f}")

The kernel value between the first and second datapoint is 0.268


In [25]:
from sklearn.svm import SVC

In [33]:
def accuracy(classifier, X, Y_target):
    return 1 - np.count_nonzero(classifier.predict(X) - Y_target) / len(Y_target)

In [34]:
svms = []
for i in classes:
    y_onevsall = np.where(y_train == i, 1, -1)
    y_onevsall_test = np.where(y_test == i, 1, -1)
    
    init_kernel = lambda x1, x2: kernel(x1, x2, params[i])
    kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, init_kernel)
    svms.append(SVC(probability=True, kernel=kernel_matrix).fit(X_train, y_onevsall))
    
    accuracy_init = accuracy(svms[i], X_train, y_onevsall)
    print(f"The accuracy of the kernel for class {i} with random parameters is {accuracy_init:.3f}")
    
    accuracy_init = accuracy(svms[i], X_test, y_onevsall_test)
    print(f"The accuracy of a kernel for class {i} with trained parameters is {accuracy_init:.3f}")

The accuracy of the kernel for class 0 with random parameters is 1.000
The accuracy of a kernel for class 0 with trained parameters is 0.952
The accuracy of the kernel for class 1 with random parameters is 0.733
The accuracy of a kernel for class 1 with trained parameters is 0.714
The accuracy of the kernel for class 2 with random parameters is 0.800
The accuracy of a kernel for class 2 with trained parameters is 0.771


In [35]:
def target_alignment(
    X,
    Y,
    kernel,
    assume_normalized_kernel=False,
    rescale_class_labels=True,
):
    """Kernel-target alignment between kernel and labels."""

    K = qml.kernels.square_kernel_matrix(
        X,
        kernel,
        assume_normalized_kernel=assume_normalized_kernel,
    )

    if rescale_class_labels:
        nplus = np.count_nonzero(np.array(Y) == 1)
        nminus = len(Y) - nplus
        _Y = np.array([y / nplus if y == 1 else y / nminus for y in Y])
    else:
        _Y = np.array(Y)

    T = np.outer(_Y, _Y)
    inner_product = np.sum(K * T)
    norm = np.sqrt(np.sum(K * K) * np.sum(T * T))
    inner_product = inner_product / norm

    return inner_product

In [36]:
def run(features_train, labels_train, features_test, labels_test, params):
    opt = qml.GradientDescentOptimizer(0.2)

    for i in range(500):
        # Choose subset of datapoints to compute the KTA on.
        subset = np.random.choice(list(range(len(features_train))), 4)
        # Define the cost function for optimization
        cost = lambda _params: -target_alignment(
            features_train[subset],
            labels_train[subset],
            lambda x1, x2: kernel(x1, x2, _params),
            assume_normalized_kernel=True,
        )
        # Optimization step
        params = opt.step(cost, params)

        # Report the alignment on the full dataset every 50 steps.
        if (i + 1) % 50 == 0:
            current_alignment = target_alignment(
                features_train,
                labels_train,
                lambda x1, x2: kernel(x1, x2, params),
                assume_normalized_kernel=True,
            )
            print(f"Step {i+1} - Alignment = {current_alignment:.3f}")
            
    # First create a kernel with the trained parameter baked into it.
    trained_kernel = lambda x1, x2: kernel(x1, x2, params)

    # Second create a kernel matrix function using the trained kernel.
    trained_kernel_matrix = lambda X1, X2: qml.kernels.kernel_matrix(X1, X2, trained_kernel)

    # Note that SVC expects the kernel argument to be a kernel matrix function.
    svm_trained = SVC(probability=True, kernel=trained_kernel_matrix).fit(features_train, labels_train)
    
    return svm_trained

In [37]:
svms_trained = []
for i in classes:
    y_onevsall = np.where(y_train == i, 1, -1)
    y_onevsall_test = np.where(y_test == i, 1, -1)
    svms_trained.append(run(X_train, y_onevsall, X_test, y_onevsall_test, params[i]))
    
    accuracy_trained = accuracy(svms_trained[i], X_train, y_onevsall)
    print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")
    
    accuracy_trained = accuracy(svms_trained[i], X_test, y_onevsall_test)
    print(f"The accuracy of a kernel with trained parameters is {accuracy_trained:.3f}")

Step 50 - Alignment = 0.371
Step 100 - Alignment = 0.426
Step 150 - Alignment = 0.462
Step 200 - Alignment = 0.481
Step 250 - Alignment = 0.503
Step 300 - Alignment = 0.518
Step 350 - Alignment = 0.525
Step 400 - Alignment = 0.525
Step 450 - Alignment = 0.534
Step 500 - Alignment = 0.533
The accuracy of a kernel with trained parameters is 1.000
The accuracy of a kernel with trained parameters is 0.990
Step 50 - Alignment = 0.085
Step 100 - Alignment = 0.089
Step 150 - Alignment = 0.088
Step 200 - Alignment = 0.097
Step 250 - Alignment = 0.098
Step 300 - Alignment = 0.104
Step 350 - Alignment = 0.111
Step 400 - Alignment = 0.117
Step 450 - Alignment = 0.120
Step 500 - Alignment = 0.112
The accuracy of a kernel with trained parameters is 0.756
The accuracy of a kernel with trained parameters is 0.771
Step 50 - Alignment = 0.135
Step 100 - Alignment = 0.148
Step 150 - Alignment = 0.151
Step 200 - Alignment = 0.155
Step 250 - Alignment = 0.164
Step 300 - Alignment = 0.171
Step 350 - Alignm

In [41]:
def softmax(x):
    return np.exp(x)/np.repeat(np.exp(x).sum(axis = -1), 3).reshape(x.shape[0], 3)

In [42]:
def score(features, labels):
    probs = np.stack([svms_trained[i].predict_proba(features)[:,1] for i in classes]).T
    predictions = softmax(probs).argmax(axis = -1)
    return sum(predictions == labels)/len(labels)
    print(probs.shape)

In [43]:
print(score(X_train, y_train))
print(score(X_test, y_test))

0.8666666666666667
0.780952380952381
